Reference: Thorn Friihwirth Slim Abdennadher Essentials of Constraint Programming 

# CSP problems  ( CLP is an authomatic and fast way to solve them). Remember that a CSP problem is fromulated like this:
- Variables:

  X = {X1, X2, X3}


- Domains:

  D(X1) = {1,2}, D(X2) = {0,1,2,3}, D(X3) = {2,3}


- Constraints

  X1 > X2 and X1 + X2 = X3 and X1 ≠ X2 ≠ X3 ≠ X1


- Solution

  X1 = 2, X2 = 1, X3 = 3 alldifferent([X1, X2, X3]) 

# CLP = Constraint Logic Programming
https://www.math.unipd.it/~frossi/gulp.pdf


Before reading this read LP notebook, because CLP is like an extension of it.
Beside the classical resolution engine of logic programming (LP), a (complete or incomplete) constraint solving system is added, able to check the consistency of
constraints of the considered sort. Moving from LP to CLP, the concept of unification is generalized to constraint solving: the relationship between a goal and
a clause (to be used in a resolution step) can be described not only via term
equations but via more general statements, i.e., constraints.
More precisely, A CLP CLAUSE IS JUST LIKE AN LP CLAUSE, EXCEPT THAT ITS BODY MAY CONTAIN ALSO CONSTRAINTS.. For example, if we can use
linear inequations over the reals, a CLP clause could be:
p(X,Y) :- X < Y+1, q(X), r(X,Y,Z).
This clause states that p(X,Y) is true if q(X) and r(X,Y,Z)
are true, and if the value of x is smaller than that of y + 1.
A constraint logic program is a logic program that contains constraints in the body of clauses. An example of a clause including a constraint is 

Also the queries can have constraints!!

From the operational point of view, in an LP resolution step, we have to check
the existence of a most general unifier between the selected subgoal and the head
of a clause. In CLP, instead, we also have to check the consistency of the current
set of constraints (called the constraint store) with the constraints in the body
of the clause. Thus two solvers are involved: unification, as usual in LP, and the
specific constraint solver for the constraints in use. To make it more efficient,
this constraint solver may be not complete, that is, it may fail to discover some
inconsistencies. While in LP a computation state consists of a goal and a substitution, in
CLP we have a goal and a constraint store (it contains among the constraints also the substitution to do unification, in form of contraints of equivalence of terms).



# # CLP calculus, formally:

- A state is a pair <G, C>, where G is the Goal (made of atoms and constraints) and C is a constraint store.

- Derivation = sequence of transitions. A goal G can be proved if there exists a derivation from <G,0> to <0,C> with C satifiable. C is the solution of the problem!!!! (What is done is like a proof of the goal for all variable evaluations that satisfy the constraints, so usually the answer is more general than the answer given by an LP, for example an answer of a CLP could be X>Y, without any specific assignation! Instead an LP would answer a possible substitution like X=2,Y=1).

- Transition = a PAIR OF PAIRS GOAL/ CONSTRAINT STORE :  <G,C> -> <G',C'>.
  The transition is described by 3 transition rules:
  
   - UNFOLD: 
     
     IF THE SELECTED ATOMIC GOAL IS A LITERAL can be replaced by the body of a fresh variant of a clause having the same predicate in the head, by adding the body of the fresh variant AND THE CONSTRAINT FOR EQUALITY OF TERMS TO THE GOAL (OR DIRECTLY TO C). 
     
     In symbols: considering the goal element p(t1,t2,..tn) and a KB clause p(t1',t2',..tn') :- B; G' is obtained by replacing the goal(the atom) with {t1=t1',t2=t2',...tn=tn' , B } (while C doesn't change OR t1=t1',t2=t2',...tn=tn' is directly added to C ).
 
  
   - SOLVE: 
     
     IF THE SELECTED ATOMIC GOAL is a constraint "c" (it can also be a complex constraints like X<=Y ^ Z=X) can be moved from the goal to the constraint store. In symbols: G'=G \ {c}, C'=C U {c}.
     THEN THE CONSTRAINT STORE C IS SIMPLIFIED : C = C'. which means that the constraint store C can be replaced with an equivalent ( and simpler) one C'. This is what is done with methods as constraint propagation. (Sometimes thanks to this can be detected the unsatisfiability of the constraint store). If the constraint store C is inconsistent backtrack. (Only the derivation is failed, but can be done bactrack and find another derivation).
   
    
  
   - FAILURE: 
   
     The UNFOLD rule can't be applied. So backtrack. (Only the derivation is failed, but can be done bactrack and find another derivation).
   
(note: some book say that they "add the equality constraint p(t1,t2,..tn)=p(t1',t2',..tn') to C, what they really mean is that they add t1=t1',t2=t2',...tn=tn' to C).
  

# CLP calculus, less formally:
The semantics of constraint logic programs can be defined in terms of a virtual interpreter that maintains a pair <G,C>  during execution. The first element of this pair is called current goal; the second element is called constraint store. The current goal contains the literals which the interpreter is trying to prove and may also contain some constraints it is trying to satisfy; the constraint store contains all constraints the interpreter has assumed satisfiable so far.

Procedure:

- Initially, the current goal is the goal and the constraint store is empty. 

- Remove the first element from the current goal and analyzing it.

- Analysis: the element is checked if it is a constraint or a literal. Then apply the appropriate transition rule.

- Backtrack: The interpreter backtracks if a failure / inconsistency is generated. The constraint store is checked for consistency every time a new constraint is added to it. In principle, whenever the constraint store is unsatisfiable the algorithm could backtrack. ( But in practice for the sake of efficiency satisfiability is checked using methods that simplify the constraint store, that is, rewrite it into an equivalent but simpler-to-solve form. These methods can sometimes but not always prove unsatisfiability of an unsatisfiable constraint store. ) WHEN YOU BACKTRACK REMEMBER TO BACKTRACK ALSO THE CONSTRAINT STORE (REMOVING THE CONSTRAINTS ADDED IN THE NODES THAT YOU'RE BACKTRACKING).

- The interpreter has proved the goal when the current goal is empty and the constraint store is not detected unsatisfiable. 

- Result/ Answer = SET OF CONSTRAINTS IN THE CONSTRAINT STORE (MAYBE SIMPLIFIED) the current set of (simplified) constraints. (constraints like X=1, Y>X..)

# Unfold non determinism:
The Unfold transition exhibits two kinds of non-determinism.

- don't-care non-determinism:
Any atomic goal can be chosen from G. It affects the length of the derivation (which can be infinite in the worst case).


- don't-know non-determinism:
Any formula of the program/KB can be chosen to which apply unfold rule. This affects the final answer/substitution.

IN PROLOG IS USED A SO CALLED SLD RESOLUTION STRATEGY: Is taken the leftmost atomic goal of G, and the upper-most formula of the KB to apply unfold. When there's a derivation failure it backtracks. This approach can be described by making explicit the non-determinism with a rule called UnfoldSplit. Note that the problem is seen as if the root node is the initial state <G,no_subs> and the leaves are the final states, which can be failure or succesful. Each path is a derivation. Each brench is a sate transition. The search done in SLD RESOLUTION is like a depth first with a selection strategy of the atomic goal to be exploded and the formula of the program to use.

# CLP calculus vs LP calculus.
While in LP we just accumulate and compose substitutions during a computation, in CLP we accumulate and simplify constraints. Like substitutions, constraints are never removed from the constraint store, therefore the information in the constraint store increases monotonically during derivations (unless are applied methods to propagate and simplify it). As in the LP calculus, we have two degrees of non-determinism in the calculus (selecting the goal and selecting the clause). Search trees are defined in the same way as in LP. Most implementations of the CLP languages also use depth-first search with chronological backtracking (SLD resolution). 
(In practice, satisfiability of the constraint store may be checked using an incomplete algorithm, which does not always detect inconsistency). CLP are like an extension of LP languages, indeed SWI-prolog is actually a CLP, but can be used as LP. From another point of view LP is like a stupid CLP which uses generate-and-test approach, instead CLP generally uses constrain-and-generate: use facts(constraints) in active manner to reduce the search space.

Don't confuse this: A=X like if it is a substitution. "=" is a constraint and it's symmetric: A=X then also X=A).

Note that also swi prolog used as LP has some operators which can be used, but to really use it as a CLP you must use a library (with this command: :-use_module(library(clpfd)) Look at this example:

- swi-prolog as LP:     
             
             min(X,Y,Z) :- X<=Y , Z=X 
             min(X,Y,Z) :- Y<=X , Z=Y
             
- swi-prolog as CLP:
                        
             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).
             
try them with these queries:  min(A,B,R).  min(1,2,R).

differences:
-  "<=" is different from "#=<": the second one is treated as a constraint the second just as an operator which gives you true or false. Indeed the constraint syntax can't be used in swi as LP. In general the constraints are expressed as in this table https://www.swi-prolog.org/pldoc/man?section=clpb, with "#" as a prefix.

- for some query the answer can be the same, but for others swi as LP has less general answers or is not even able to compute them!!



# How to express constraints:

The use of "or", "not" is not discussed in LP. You must use only "<-" (":-" in prolog) and "and"("," in prolog). For example if you want to write p(X):- p(a) V p(b) you must write this:  p(X):-p(a) and in another line this  p(X):-p(b). For as regars the "not" there's a built in predicate called "not".

to express contraints in Finite domain look this: https://www.swi-prolog.org/pldoc/man?section=clpfd. Note that the operators like +,-.. have not the "#" like #=<,#=,...

IN CLP THERE ARE THESE BOOLEAN OPERATORS: https://www.swi-prolog.org/pldoc/man?section=clpb-exprs


# DECLARATIVE SEMANTIC:
Is less interessant than LP semantic. Just for curiosity:
Let <G', C> be a state which occurs in a derivation starting with <G, true>. The formula Exist x(G'^C) is the logical reading (= the meaning= exists a set of variables which make true G' and the constraints in C) of the state, where x stands for the variables which occur in G' or C but not in G.  An answer (constraint) of a goal G can be defined as the logical reading of a final state of a derivation started with <G,T>.

# soundness and completenes:
Similar to LP, but instead of CET we talk about CT (constraint theory, which actually is an extension of CET, which is simply a set of clauses useful to guarantee some property of the program, for instance to give the "=" constraint the property of simmetry..). Remember that soundness and completeness are property of the language (not of CLP in general!), because it depends on the strategy used. Let's see what does it mean to be sound and complete for a CLP language.

For succesful derivations:

# Soundness:
If G has successful derivation with answer constraint C, then P↔∪CT |= ∀(C → G). 

# Completeness:
If P↔∪CT |= ∀(C → G) and C is satisﬁable in CT, then there are successful derivations for G with answer constraints C1,...,Cn s.t. CT |= ∀(C → (C1 ∨...∨Cn)). 


# Soundness and Completeness for failed derivations:
P↔∪CT |= ¬∃G if and only if each fair derivation starting with <G,T> fails ﬁnitely.


# Example TO BE READ CAREFULLY:
Formally, constraint logic programs are like regular logic programs, but the body of clauses can contain constraints, in addition to the regular logic programming literals. ( ":-" means <-)

  B(X,1):- X<0.
  B(X,Y):- X=1, Y>0.
  A(X,Y):- X>0, B(X,Y).
  
query: A(X,1).
Like in regular logic programming, evaluating a goal such as A(X,1) requires evaluating the body of the last clause with Y=1. 

Like in regular logic programming, this in turn requires proving the goal B(X,1) BUT CONTRARY TO REGULAR LOGIC PROGRAMMING IT REQUIRES ALSO TO SATISFY X>0, the constraint in the body of the last clause.

CAN HAPPEN THAT THE SATISFACTION OF A CONSTRAINT CANNOT BE DETERMINED IN THE MOMENT IN WHICH IT'S ENCOUNTERED. In this case, for example, the value of X is not determined when the last clause is evaluated. As a result, the constraint X>0 is not satisfied nor violated at this point. 

SO WHAT IS IT DONE ABOUT THE CONSTRAINT?
Is it checked (postponed as a goal) after B(X,1) evaluation? NO:

The interpreter STORES THE CONSTRAINT X>0 and then proceeds in the evaluation of B(X,1); this way, the interpreter can detect violation of the constraint X>0 DURING THE EVALUATION of B(X,1) (which can be a long path of the search tree), and BACKTRACK IMMEDIATELY if this is the case, rather than waiting for the evaluation of B(X,1) to conclude and consider only at the end of it the constraint X>0. (This is why in the program is better to have constraints before predicates!!).

In general, the evaluation of a constraint logic program proceeds like for a regular logic program, but constraints encountered during evaluation are placed in a SET called "CONSTRAINT STORE". As an example, the evaluation of the goal A(X,1) proceeds by evaluating the body of the first clause with Y=1; this evaluation adds X>0 to the constraint store and requires the goal B(X,1) to be proved. While trying to prove this goal, the first clause is applicable, but its evaluation adds X<0 to the constraint store. This addition makes the constraint store unsatisfiable, and the interpreter BACKTRACKS REMOVING THE LAST ADDITION TO THE CONSTRAINT STORE. The evaluation of the second clause adds X=1 and Y>0 to the constraint store. Since the constraint store is satisfiable and no other literal is left to prove, the interpreter stops with the solution X=1, Y=1.


# 1) Example with min predicate:
Try to write the program/KB which given the query min(X,Y,Z) gives you Z= the minimum between X,Y. REMEMBER THAT IN LP AND CLP YOU CAN'T USE THE 'OR', BUT ONLY 'AND' (and constraints in CLP, like =, !=, >=,<, +...)
( ":-" means <-)

Answer:

             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).
    
Now show how the interpreter reach the solution for this goal:  min( 1,2,R) (so the initial state=  <min( 1,2, R), true>), describing the computation as a transition sys. (and also represent the SLD RESOLUTION tree)

Answer:

1) min(1,2,R) is a literal so the transition rule Unfold must be applied. The equality equation min(X,Y,Z)=min(1,2,R) creates the constraints X=1,Y=2,R=Z to be added to the constraint store, and to the goal store is added the body of the rule, X<=Y ^ Z=X . 

So:  <G: min( 1,2, R), C: true>    ->   < G: (X<=Y ^ Z=X ) , C: (X=1,Y=2,R=Z)>

2) now the first (and only) goal in the goal store is a constraint: X<=Y ^ Z=X. It's a constraint so it must be applied the transition rule SOLVE. It says that the constraint must be added to the constraint store C. Then the Constraint store can be simplified and or propagated. X<=Y is satisfied BECAUSE X=1 ^ Y=2, THEN I CAN SAY THAT Z=X AND R=Z INTRODUCE THE NEW CONSTRAINT R=X AND THANKS TO X=1 I CAN SAY R=1. 

SO : <G: (X<=Y ^ Z=X ), C: (X=1,Y=2,R=Z) >   ->   <G: true ,C: (R=1, X<=Y, Z=X ,X=1,Y=2,R=Z )> WHICH IS A FINAL STATE WITH A CONSTRAINT STORE SATISFIABLE SO IT IS A SUCCESFUL FINAL STATE. THE SOLUTION IS THE CONSTRAINT STORE ITSELF. 

BUT FOR READABILITY IN THESE EXERCISES YOU CAN SAY THAT THE ANSWER IS THE CONSTRAINT SOLVER OF WHICH YOU CONSIDER ONLY THE CONSTRAINTS WHICH INVOLVE VARIABLES OF THE INITIAL GOAL. IN THIS EXAMPLE THE INITIAL GOAL HAS ONLY R, SO THE ANSWER IS R=1.


note: 
- in this exercise was a concidence that when we had a goal it was only a literal or only a constraint. Usually you have a goal which is a conjunctive clause made by constraints and literals; the first literal/constraint determines the rule to be applied. (can be confusing that we call goal both the conjunctive clause (which more precisely should be called goal store) and each literals and constraints which compose it).
- for siplicity we didn't write the clauses as fresh variants at each step but it should be done.
- how the constraint solver is simplified actually depends on the particular constraint solver of the language, which also depends on the Domain considered, so actually you can just reason on C as you prefer if is not given a specification.


# 2) Example with min predicate:
Like the previous exercise but suppose to invert the two rules of the KB (useful to see the effect of "don't know non determinism" which says that changing the order of the formulas in the KB the answer could change, in this case it doesn't change.):

             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             
    
 initial goal = min( 1,2,R).
 
Answer:
1) min(1,2,R) is a literal so it is tried the equality with the head of the first rule: min(X,Y,Z)=min(1,2,R) which is possible thanks to the constraints X=1,Y=2,R=Z which are added to the constraint store. The body of the rule is added to the goal, So:
< G: min( 1,2, R), C: true >     ->    < G: (Y<=X ^ Z=Y ) , C: (X=1,Y=2,R=Z) >.

2) now the first (and only) goal in the goal store is a constraint:  Y<=X ^ Z=Y, so it must be added to the constraint store which becames C:( Y<=X ,Z=Y , X=1,Y=2,R=Z) which is unsatisfiable because I have Y<=X where Y=2 and X=1. So it must be done backtrack to the last choice point: instead of the second rule the first one must be used to apply the Unfold rule. The derivation becames the same done in the previous exercise.


# 3) Example with min predicate:
KB:
    
             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).
             
goal: min (A, 2 , 1) 

answer:

1) min(A,2,1) is a literal so it tries to be equal to the head of the first rule: min(X,Y,Z) = min (A, 2 , 1) so we have these constraints to add to the constraint store: A=X, Y=2, Z=1. The body of the rule is added to the goal store.
So : < G: min (A, 2 , 1),C: True >     ->     < G: (X<=Y ^ Z=X ), C:(  A=X, Y=2, Z=1) >

2) now the first(and only) goal is a constraint: X<=Y ^ Z=X. So it must be added to the constraint store: X<=Y, Z=X ,  A=X, Y=2, Z=1. Now it must be simplified/ propagated and the information that we can infer is A=1. The constraint store is satisfiable and the goal store is empty, so the whole constraint store is actually the solution. But for readability I say that the answer is A=1 (bacause is the variable we care about).






# 4) Example with min predicate, Failure case:
KB:
    
             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).

The goal min (A, 2,3)

answer:

<min(A,2,3),T> -> < G: (X<=Y ^ Z=X) , C:( X=A, Y=2, Z=3) > -> <G: True, C:( X<=Y , Z=X ,  X=A, Y=2, Z=3) > which is not satisfiable because it leads to 3<=2, so it's a failure. So it backtracks and does this:  <G: min(A,2,3), C: True > -> < G: (Y<=X ^ Z=Y) , C:( X=A, Y=2, Z=3) > -> <G:True, C: (Y<=X, Z=Y ,  X=A, Y=2, Z=3 )> which is not satisfiable because it leads to Y=3=2 which is not satisfiable. So prolog hasn't even one succesful derivation -> ERROR/FAILURE.

# 5) Example with min predicate, general answer:
KB:
    
             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).

goal: min(A,A,B)

answer1:

<G: min(A,A,B), C: True> -> <G: (X<=Y ^ Z=X) , C:( X=A,Y=A,Z=B) > -> <G: True, C:( X<=Y , Z=X , X=A,Y=A,Z=B) > which gives us these informations about the initial goal variables: A<=A, B=A  which are both satisfiable. so the constraint store is satisfiable and the answer is the constraint store, but we care especially about this: B=A.
(using first the second rule in this case it leads to the same answer).

# 6) Example with min predicate, ORDER MATTERS ("don't know non determinism": if you change the selection order of the KB clauses, the answer could change):
KB:
    
             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).

goal: min(A,B,R)

answer1:
<G:min(A,B,R),C:True> -> <G:(X<=Y ^ Z=X ), C:( X=A, Y=B, Z=R) > ->  <G:True, C:( X<=Y, Z=X , X=A, Y=B, Z=R)>, this is a satisfiable constraint store, so the solution is itself, but we especially care about this information that can be infered: R=A, A<=B. 

answer2:
suppose to use the second rule before:

<G: min(A,B,R),C: True> -> <G:(Y<=X ^ Z=Y), C:(X=A, Y=B, Z=R)> -> <G:True, C:( Y<=X ,Z=Y, X=A, Y=B, Z=R)> which is satisfiable, and the relevant information is: R=B, B<=A.

SO NOTE THAT THE ORDER OF THE SELECTION OF THE RULES CAN CHANGE THE ANSWER.



# 7) Example with min predicate, LOGICAL READING:
KB:
    
             :-use_module(library(clpfd)).
             min(X,Y,Z) :- (X #=< Y) , (Z #=X).
             min(X,Y,Z) :- (Y #=< X) , (Z #=Y).

if it is asked the logical reading of the KB, first of all remember that logical reading means to give the semantic meaning of something. In this case the KB is saying: ∀ X,Y,Z(min(X,Y,Z) <-> X<=Y ^ Z=X  or Y<=X ^ Z=Y).
It could be asked also for the states.

# HANDLING CONSTRAINTS
We said that for the transition rules discussed the constraint store can only increase. That's True but there's a specific part of the interpreter which handles constraints. It can do simplifications like X<=Y ^ X>=Y   becomes   X=Y. Or propagations: constraints are added, for instance to reduce the domain of a variable. (this is done by a specific language for handling constraints, the most used is CHR (Constraints Handling Rules)).





# DIFFERENT CLP LANGUAGES 
CLP is not a programming language, but a programming paradigm, which is parametric with respect to the class (=kind) of constraints used in the language. Working with a particular CLP language means choosing a speciﬁc class of constraints (for example, ﬁnite domains, linear, or arithmetic) and a suitable constraint system (so also a particular constraint handlers) for that class. Let's see the more diffused:


- TREE DOMAIN: terms= variable values. The only constraints considered are equalities and disequalities between terms, like t1=t2. Equality constraints on terms can be simplified/ solved via unification. 

- REAL DOMAIN : terms = real expressions over variables and real constants. Each variable can only take a real number as a value.
  Reals and function symbols can be combined, leading to terms that are expressions over reals and function symbols applied       to other terms.  In this case, variables ranges over real numbers and terms. (Equality of two terms can be simplified           using the rules for tree terms if none of the two terms is a real expression)
      
- FINITE DOMAIN: variables are constrained to take their value from a given, finite set. Choosing integers for values allows for arithmetic expressions as constraints. Constraint propagation proceeds by removing values from the sets of possible values that do not participate in any (partial) solution. (e.g X in 0..5. means X domain is [0,1,2,3,4,5]). THIS IS THE MOST POPULAR kind of language. Usually, the consistency check is based on some kind of constraint propagation, such as arc-consistency, some weaker version, like bound-consistency, or, more rarely, path-consistency. 

!!!!!!!!!!!!!!!!!!!!!SWI-PROLOG IS ONE OF THESE CLP Finite Domain  LANGUAGES!!!!!!!!!!!!!!!!!!!!!!!

# CLP(FD) = CLP FINITE DOMAIN
main properties:
- They have a speciﬁc mechanism for deﬁning the initial ﬁnite domains of the variables: usually as an interval over the integers. For example, a typical CLP(FD) syntax to say that the domain of variable x contains all integers between 1 and 10 is X in [1..10], or X::[1,10]

- A built-in predicate called "LABELING" (or "label", see them on swi prolog library) deﬁned over a list of variables, and which ﬁnds values for them such that all constraints in the current store are satisﬁed. The labeling predicate provides a mechanism to generate solutions, that is, variable assignments that satisfy all accumulated constraints. More precisely, this predicate triggers backtracking search over a set of variables.

EXAMPLE:
p(X,Y,Z) :- [X,Y,Z] ins [1,10], X + Y = 9 - Z,label([X,Y,Z]).

It deﬁnes a problem with three ﬁnite domain variables (x, y,z), each with domain containing the integers from 1 to 10, and sets a constraint over them (x+y =9−z). After this, it triggers backtracking search via predicate labeling.

Query:  :- p(X,Y,Z). Answer: ANY POSSIBLE INSTANTIATION of the three variables over their domains which satisﬁes the constraint x + y =9−x. 

Notice that without labeling, this same goal would return just the new domains obtained after applying constraint propagation :  [1..7].  


The clause above presents the typical shape of a CLP(FD) program: ﬁrst the variable domains are speciﬁed, then the constraints are imposed, and ﬁnally the backtracking search is invoked via a labeling predicate. A CLP(FD) program can consist of many clauses, but the overall structure of the program always reﬂects this order, which refers to a methodology called constrain and generate, where ﬁrst variables are constrained and only later (when the domains are smaller) backtracking search is invoked. This corresponds to applying constraint propagation prior to search and therefore avoiding early some dead-ends. 

more info for CLP(FD):

tutorial =http://www.pathwayslms.com/swipltuts/clpfd/clpfd.html

CLP(FD) SWI-PROLOG library and examples (also sudoku and n-queens) = https://apps.nms.kcl.ac.uk/reactome-pengine/documentation/man?section=clpfd

labeling predicate:https://apps.nms.kcl.ac.uk/reactome-pengine/documentation/man?predicate=labeling/2


# example: factorial number in CLP(FD): 

:- use_module(library(clpfd)).
factorial(0,1).
factorial(N,R):-  N1#=N-1, factorial(N1, Rp), R #= N * Rp.

BUT THE CLP PROGRAMS ARE MORE EFFICIENT IF YOU PUT THE CONSTRAINTS BEFORE, and to make it more robust add the condition N#>0:
     
:- use_module(library(clpfd)).
factorial(0,1).
factorial(N,R):-  N#>0, N1#=N-1, R #= N * Rp, factorial(N1, Rp).

The two programs do not even have the same termination properties in all cases. Reordering goals (and clauses) can significantly impact the performance of Prolog programs, and you are free to try different variants if you use declarative approaches. Moreover, since all CLP(FD) constraints always terminate, placing them earlier can at most improve, never worsen, the termination properties of your programs. 

# Crypto-arithmetic Puzzle – Send More Money:
Replace distinct letters by distinct digits (so between 0..9), M and S can't be zero (because at the beginning of the numbers).
The problem is usually expressed in this form:

      S E N D +
   
      M O R E =
   
    M O N E Y

(Example of a solution: [S,E,N,D,M,O,R,Y] = [9,5,6,7,1,0,8,2])


To solve it with CLP approach the only problem is to model the problem: write the KB/program with the constraints to rapresent it.

- You can model the domains with this:

  puzzle_problem(....):- Vars = [S,E,N,D,M,O,R,Y], Vars ins 0..9, .....

  Note that the command ins defines the domain 0..9 for each variable into the list Vars.
 
 - Then we want also to add the constraint M and S can't be zero.
  This is done like this:  M #\= 0, S #\= 0.


- We also want to add the constraint that all variables must be different. That is possible thanks to another built in constraint:

  all_distinct(Vars)
  
- Now how can you add the constraints about the sum of the variables SEND + MORE in that specific order to create MONEY? If you reason about it each variable must have a digit, but you can specify the position in the sum exactly as the sum between number is done: associate to each variable a position by multiplying it with 10^x :
  S*1000 + E*100 + N*10 + D     +    M*1000 + O*100 + R*10 + E #=  M*10000 + O*1000 + N*100 + E*10 + Y. Note that in SWI-Prolog  the constraint operators are indicated using "#".

  
- Finally we can make the interpreter find the assignation which are possible to give to the variables by using the built in predicate:   label(Vars).

The whole program look like this:

:- use_module(library(clpfd)).               (= the library to use CLP(FD))

puzzle([S,E,N,D] + [M,O,R,E] = [M,O,N,E,Y]) :-  
        
        Vars = [S,E,N,D,M,O,R,Y],     
        Vars ins 0..9,  
        M #\= 0, S #\= 0,   
        all_distinct(Vars),          
                  S*1000 + E*100 + N*10 + D +                       
                  M*1000 + O*100 + R*10 + E #=                  
        M*10000 + O*1000 + N*100 + E*10 + Y,        
         
		label(Vars).
        
to the query puzzle(X) what is the answer?
X = ([9, 5, 6, 7]+[1, 0, 8, 5]=[1, 0, 6, 5, 2]) ;  which if you reason on it means [S,E,N,D,M,O,R,Y] =  [9,5,6,7,1,0,8,2]
So we've solved the problem by using CLP(FD)!!! the only thing we had to do was to model the problem.
If you remove the predicate label(Vars), the answer is: 
M = 1, O = 0, S = 9, D in 2..8, R in 2..8, Y in 2..8, N in 5..8, E in 4..7, 91*E+D+10*R#=90*N+Y, all_distinct([9, E, N, D, 1, 0, R, Y]).

# Crypto-arithmetic Puzzle – Send More Money, without the use of the "ins" constraint:
the only difference is that you need to create a predicate which does this by exploiting the constraint "in":
this must be put in the Body of the predicate puzzle: generate_domains(Vars,0..9), it will work like a call to the predicate generate_domains (indeed the body of the rule puzzle became the new goal).

Outside the puzzle predicate instead write what generate_domains must do:

generate_domains([H|T],D):- H in D, generate_domains(T,D).

But if you do this when T is the empty list it raises a failure, so you must handle that case too: 

generate_domains([],_). Which means that is the list given in input is empty it returns T, so it stops the recursion.

So the whole program is:

:- use_module ( library ( clpfd )).

:- use_module(library(clpfd)).

puzzle([S,E,N,D] + [M,O,R,E] = [M,O,N,E,Y]) :-

    Vars = [S,E,N,D,M,O,R,Y],
    generate_domains(Vars ,0..9) ,
    S #\= 0, M #\= 0,
    all_distinct(Vars),
    1000* S + 100* E + 10*N + D
    + 1000* M + 100* O + 10*R + E
    #= 10000* M + 1000* O + 100* N + 10*E + Y,
    labeling([] ,[S,E,N,D,M,O,R,Y]).

generate_domains([] ,_).
generate_domains([H|T],D) :- H in D, generate_domains(T,D).





# Contraint solvers:
A CONSTRAINT SOLVER IS A SYS WHICH TAKES AS INPUT A CONSTRAINT (OR A CONSTRAINT STORE) AND OUTPUTS IF IT IS SATISFIABLE, OR NOT SATISFIABLE, OR UNKNOWN. (A solver is complete if it always answers true or false. (never unknown))

- Valuation: an assignment of values (of a suitable domain) to variables.
- Solution: valuation which satisfies constraint
- Satisfiable: constraint has a solution
- Equivalent constraints: same set of solutions

To play with constraints imagine that at the beginning each variable has as domain all the numbers. Then reduce the domain by propagation of the constraints.
     
   X <= 3 ^ Y = X +1  satisfiable
   
   X <= 3 ^ Y = X +1 ^ Y >= 6  unsatisfiable
  
- concept of SOLVED FORM:
   X=Y ∧ X=Z not in solved form, because X occurs twice on the left hand side of the constraints.
   X=Y ∧ Y =Z is in solved form
   X=Z ∧ Z=Y is in solved form
   
A constraint solver answers the satisfaction problem of a constraint (or a set of constraints).
How does a constraint solver works? The stupid solution could be to try all the possible evaluations, with reals domains is not even possible. More smart solution can be found, but are domain dependent:
   


#  Constraint solvers for FINITE domain:
All the things like arc consistency...


# Constraint solvers for REAL domain: 
Gauss-Jordan elimination:

example: contrainst=   X+1 = 2Y + Z  , Z - X = 3

Replace X = 2Y + Z -1    so obtain  X =  2Y + Z -1  ,  Z -  (2Y + Z -1) = 3.    
Now use the equation to replace Y:   Z -  (2Y + Z -1) = 3 .  It's found Y = -1.
So you obtain X = Z-3 , Y = -1.  This is in SOLVED FORM, and it's satisfiable. To find a solution you can do like this:
at first assign the value of a variable on the right side of an equation (also called parametric variable), then evaluate another parametric variable and so on.


The result may rely on order of variables and expressions.
Just remember that order of the constraints in a Program matters: it can change the length of the derivation (in the worse case a certain order can lead to a non terminatio).

# CONSTRAINT SOLVERS ON TREE DOMAIN:


# TREE CONSTRAINTS

- Tree constraints represent structured data
- Tree constructor: character string
- cons, node, null, widget, f
- Constant: constructor or number
- Tree:
  - A constant is a tree
  - A constructor with a list of > 0 trees is a tree
  - IT FOLLOWS THE RULE tree(ROOT, LEFT_MOST_CHILD,...., RIGHT_MOST_CHILD):

example: order( part(77665, widget(red, moose) ), quantity(17), date(3, feb, 1994))

<img src= "tree clp.jpg"  height=50% width=50%>

# Terms 
- A term is a tree with variables replacing subtrees
- Term:
   - A constant is a term
   - A variable is a term
   - A constructor with a list of > 0 terms is a term
   - Term equation: s = t (s,t terms)
   
# Tree Constraint Solving
- Assign trees to variables so that the terms are identical
- consider the constraint on these tree: cons(R, cons(B, nil)) = cons(red, L)   
  the solution is done by {R=red, L=cons(blue,nil), B=blue}
- Similar to Gauss-Jordan
- Starts with a set of term equations C and an empty set of term equations S
- Continues until C is empty or it returns false

for the constraint like cons(Y, nil)= cons(X,Z) ^ Y = cons(a,T)
- Remove equation c from C, then if c is:
   - x=x: do nothing
   - f(s1,..,sn)=g(t1,..,tn): return false
   - f(s1,..,sn)=f(t1,..,tn):
        - add s1=t1, .., sn=tn to C
   -  t=x (x variable): add x=t to C
   - x=t (x variable): add x=t to S and:
     - substitute t for x everywhere else in C and S


<img src= "tree constraint solver clp.jpg"  height=50% width=50%>

